In [25]:
import numpy as np
import os
import pickle
import torch
from typing import List, Dict, Tuple
import pandas as pd
import torch.nn.functional as F
from genReLiTu import plot_detection_heatmaps_3x4, generate_data_1dimension
import sklearn.metrics.pairwise as smp
from collections import defaultdict
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import LocalOutlierFactor
from sklearn.metrics import matthews_corrcoef, roc_auc_score
from scipy.cluster.hierarchy import linkage, fcluster
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from PIL import Image
from pdf2image import convert_from_path

In [26]:
dataset = "fmnist"

iles = {
    "MR": {
        "flame": "10020337",
        "fltrust": "10022140",
        "foolsgold": "10271625",
        "secfft": "10292241",
    },
    "EDGE_CASE": {
        "flame": "10020911",
        "fltrust": "10030309",
        "foolsgold": "10280055",
        "secfft": "10300448",
    },
    "NEUROTOXIN": {
        "flame": "10021255",
        "fltrust": "10030648",
        "foolsgold": "10280559",
        "secfft": "10300840",
    },
}

attack_defense_data = {
    "MR": {
        "FLAME": "FL_Backdoor_CV/2024-10-02_03-37-50/",
        "FLTRUST": "FL_Backdoor_CV/2024-10-02_21-40-11/",
        "FOOLSGOLD": "FL_Backdoor_CV/2024-10-27_16-25-41/",
        "SECFFT": "FL_Backdoor_CV/2024-10-29_22-41-06/",
    },
    "EDGE_CASE": {
        "FLAME": "FL_Backdoor_CV/2024-10-02_09-11-28/",
        "FLTRUST": "FL_Backdoor_CV/2024-10-03_03-09-36/",
        "FOOLSGOLD": "FL_Backdoor_CV/2024-10-28_00-55-46/",
        "SECFFT": "FL_Backdoor_CV/2024-10-30_04-48-52/",
    },
    "NEUROTOXIN": {
        "FLAME": "FL_Backdoor_CV/2024-10-02_12-55-32/",
        "FLTRUST": "FL_Backdoor_CV/2024-10-03_06-48-00/",
        "FOOLSGOLD": "FL_Backdoor_CV/2024-10-28_05-59-22/",
        "SECFFT": "FL_Backdoor_CV/2024-10-30_08-40-55/",
    },
}

In [27]:
# for attack, attack2data in attack_defense_data.items():
#     true_labels = []
#     labels_get_by_Euclid = []
#     labels_get_by_manhattan = []
#     labels_get_by_cosine = []
#     labels_get_by_chi_square = []
#     for denfense, data_folder in attack2data.items():
#         if denfense != "AVG":
#             continue
#         participant_file_name = f"{data_folder}participants/participants.csv"
#         participants = np.genfromtxt(
#             participant_file_name, delimiter=",", dtype=None, encoding="utf-8"
#         )
#         participants = participants[1:].T
#         for i in range(len(participants) - 1):
#             file_name = f"{data_folder}model_updates/{dataset}_{attack}_{i}.pkl"
#             if not os.path.exists(file_name):
#                 print(f"File {file_name} not found")
#                 continue
#             with open(file_name, "rb") as file:
#                 model_updates = pickle.load(file)


# roundNum = 10
# dirPath = '../NormalRun/FL_Backdoor_CV/resultWithTime/2024-09-12_23-26-43-avg_NEUROTOXIN-fmnist/'
# pklName = os.path.join(dirPath, f'model_updates/fmnist_NEUROTOXIN_{roundNum}.pkl')
# participantFilePath = os.path.join(dirPath, 'participants/participants.csv')
# # participantFileDF = pd.read_csv(participantFilePath)
# # # 提取参与者数组（第一列）和轮次（从第二列开始的部分）
# # clients = participantFileDF.iloc[1:, 0].tolist()  # 客户端列表
# # roundsArray = participantFileDF.iloc[1:, 1:].to_numpy()  # 轮次数据（50行x30列）
# # print(roundsArray)
# participants = np.genfromtxt(
#     participantFilePath, delimiter=",", dtype=None, encoding="utf-8"
# )
# participants = participants[1:].T
# participants_thisRound = participants[roundNum]
# print(participants_thisRound)

# with open(pklName, 'rb') as f:
#     update: Dict[str, torch.Tensor] = pickle.load(f)
# # print(update)
# # weight0 = update['base_model.model.vision_model.encoder.layers.0.self_attn.k_proj.lora_A.default.weight']
# # weight0.shape  # torch.Size([50, 12288])
# userUpdates = [torch.empty(0) for _ in range(50)]
# for layerKey, values in update.items():
#     # print(layerKey)
#     # print(values)
#     # print(values.shape)  # torch.Size([50, 12288])
#     for i in range(values.shape[0]):
#         trueUser = participants_thisRound[i]
#         userUpdates[trueUser] = torch.cat((userUpdates[trueUser], values[i].cpu()), 0)
# print(userUpdates[0].shape)

def loadPkl(roundNum: int, dirPath: str) -> Dict[str, torch.Tensor]:  # 把pkl变成{'key1': [[user3], [user5]]}
    # dirPath = '../NormalRun/FL_Backdoor_CV/resultWithTime/2024-09-12_23-26-43-avg-fmnist_NEUROTOXIN/'
    pklPrefix = os.path.basename(os.path.normpath(dirPath)).split('-')[-1]
    pklName = os.path.join(dirPath, f'model_updates/{pklPrefix}_{roundNum}.pkl')
    with open(pklName, 'rb') as f:
        update: Dict[str, torch.Tensor] = pickle.load(f)
    return update

def getParticipants(roundNum: int, dirPath: str) -> np.ndarray:  # 获取参与者列表
    participantFilePath = os.path.join(dirPath, 'participants/participants.csv')

    # 读取参与者数组
    participants = np.genfromtxt(
        participantFilePath, delimiter=",", dtype=None, encoding="utf-8"
    )
    participants = participants[1:].T
    participants_thisRound = participants[roundNum]  # 获取当前轮次的参与者
    return participants_thisRound

def get_all_user_updates(roundNum: int, dirPath: str) -> torch.Tensor:  # 把pkl变成[[user1展平(拼接)后的结果], [user2], ...]
    # 设置轮次编号和文件路径
    # roundNum = 10
    # dirPath = '../NormalRun/FL_Backdoor_CV/resultWithTime/2024-09-12_23-26-43-avg-fmnist_NEUROTOXIN/'
    participants_thisRound = getParticipants(roundNum, dirPath)
    # print(participants_thisRound)

    # 加载 .pkl 文件
    update = loadPkl(roundNum, dirPath)

    # 初始化一个列表来存储每个客户端的更新，使用 GPU 张量
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    userUpdates = [torch.empty((0,), device=device) for _ in range(50)]  # 假设50个客户端

    # 遍历每个层的更新
    for layerKey, values in update.items():
        # 将每个值移到 GPU 上
        values = values.to(device)
        for i in range(values.shape[0]):  # 遍历每个客户端的梯度
            trueUser = participants_thisRound[i]  # 获取当前轮次的客户端ID
            # 如果为空张量
            if userUpdates[trueUser].numel() == 0:
                userUpdates[trueUser] = values[i]  # 直接赋值
            else:
                userUpdates[trueUser] = torch.cat((userUpdates[trueUser], values[i]), 0)  # 在GPU上拼接

    userUpdates = torch.stack(userUpdates)  # 转换为张量
    # # 打印第一个用户的更新形状来检查结果
    # print(userUpdates[0].shape)
    return userUpdates

In [28]:
def cosine_similarity_matrix(tensor: torch.Tensor) -> torch.Tensor:
    """
    计算一个二维 tensor 中每两个向量之间的余弦相似度矩阵。

    参数:
    - tensor (torch.Tensor): 形状为 (N, D) 的二维张量，N 是客户端数量，D 是特征维度。

    返回:
    - similarity_matrix (torch.Tensor): 形状为 (N, N) 的张量，每个元素表示两个向量之间的余弦相似度。
    """
    # 归一化每个向量
    normalized_tensor = F.normalize(tensor, p=2, dim=1)
    
    # 计算余弦相似度矩阵
    similarity_matrix = torch.mm(normalized_tensor, normalized_tensor.T)
    
    return similarity_matrix


In [29]:
# userUpdates_avg_NEUR = get_all_user_updates(10, '../NormalRun/FL_Backdoor_CV/resultWithTime/2024-09-12_23-26-43-avg-fmnist_NEUROTOXIN/')
# print(userUpdates_avg_NEUR.shape)
# print(userUpdates_avg_NEUR)
# userUpdates_avg_MR = get_all_user_updates(10, '../NormalRun/FL_Backdoor_CV/resultWithTime/2024-09-13_12-02-44-avg-fmnist_MR/')

# similarity_matrix_avg_NEUR = cosine_similarity_matrix(userUpdates_avg_NEUR)
# print(similarity_matrix_avg_NEUR.shape)
# similarity_matrix_avg_MR = cosine_similarity_matrix(userUpdates_avg_MR)
# # print(similarity_matrix_avg_MR.shape)
# plot_detection_heatmaps(similarity_matrix_avg_NEUR, similarity_matrix_avg_MR)

In [30]:
# foolsgold
# ChangeFrom https://github.com/LetMeFly666/SecFFT/blob/706bb287c3b00f6143e2190edc74714ed88f3532/getResult/FL_Backdoor_CV/roles/aggregation_rules.py#L203C1-L241C25
# foolsgold自身并没有确认哪些是恶意客户端，而是根据权重聚合
# def foolsgold_identify_malicious_clients(wv: np.ndarray, threshold: float = 0.25) -> np.ndarray:
#     """
#     根据权重向量识别潜在的恶意客户端。

#     参数:
#     - wv (np.ndarray): 客户端的权重向量。
#     - threshold (float): 判断恶意客户端的阈值，权重低于此值的客户端将被视为恶意客户端。
    
#     返回:
#     - malicious_clients (np.ndarray): 被识别为恶意的客户端索引。
#     """
#     return np.where(wv < threshold)[0]
def foolsgold_identify_malicious_clients(wv: np.ndarray) -> np.ndarray:
    kmeans = KMeans(n_clusters=2, random_state=0).fit(wv.reshape(-1, 1))
    clusters = kmeans.labels_
    unique, counts = np.unique(clusters, return_counts=True)
    max_cluster = unique[np.argmax(counts)]
    user_labels = np.zeros(len(clusters), dtype=int)
    user_labels[clusters == max_cluster] = 1
    return np.where(user_labels == 0)[0]

# 返回 聚合后的全局模型、恶意客户端索引、余弦相似度矩阵
# def foolsgold(model_updates: Dict[str, torch.Tensor]) -> Tuple[Dict[str, torch.Tensor], np.ndarray, np.ndarray]:
def foolsgold_oneRound(roundNum: int, dirPath: str) -> Tuple[Dict[str, torch.Tensor], np.ndarray, torch.Tensor]:
    model_updates = loadPkl(roundNum, dirPath)
    participants_thisRound = getParticipants(roundNum, dirPath)
    
    keys = list(model_updates.keys())
    last_layer_updates = model_updates[keys[-2]]
    K = len(last_layer_updates)
    cs = smp.cosine_similarity(last_layer_updates.cpu().numpy()) - np.eye(K)  # 减去对角线为1的单位矩阵，使得自身与自身的相似度为0
    maxcs = np.max(cs, axis=1)
    # === pardoning(赦免) ===
    for i in range(K):
        for j in range(K):
            if i == j:
                continue
            if maxcs[i] < maxcs[j]:
                cs[i][j] = cs[i][j] * maxcs[i] / maxcs[j]

    alpha = np.max(cs, axis=1)
    wv = 1 - alpha
    wv[wv > 1] = 1
    wv[wv < 0] = 0

    # === Rescale so that max value is wv ===
    wv = wv / np.max(wv)
    wv[(wv == 1)] = .99

    # === Logit function ===
    wv = (np.log(wv / (1 - wv)) + 0.5)
    wv[(np.isinf(wv) + wv > 1)] = 1
    wv[(wv < 0)] = 0

    malicious_clients = foolsgold_identify_malicious_clients(wv)
    malicious_clients = participants_thisRound[malicious_clients]
    print(f"识别出的恶意客户端索引: {malicious_clients}")

    # === calculate global update ===
    global_update = defaultdict()
    for name in keys:
        tmp = None
        for i, j in enumerate(range(len(wv))):
            if i == 0:
                tmp = model_updates[name][j] * wv[j]
            else:
                tmp += model_updates[name][j] * wv[j]
        global_update[name] = 1 / len(wv) * tmp
    print(wv)
    if True:  # 使用余弦相似度作为热力图依据
        cs = smp.cosine_similarity(last_layer_updates.cpu().numpy())  # 这里就不再减去自身了
        cs_rearranged = np.zeros((len(participants_thisRound), len(participants_thisRound)))
        for i, user_i in enumerate(participants_thisRound):
            for j, user_j in enumerate(participants_thisRound):
                cs_rearranged[user_i][user_j] = cs[i][j]
        cs_tensor = torch.from_numpy(cs_rearranged)
    else:  # 使用聚合参数wv作为热力图依据。这样直接很多1，太明显了
        cs_tensor = torch.zeros((len(participants_thisRound)))
        for i, user_i in enumerate(participants_thisRound):
            cs_tensor[user_i] = wv[i]  # tensor([0.2348, 0.2348, 0.8740, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000])

    return global_update, malicious_clients, cs_tensor

# 组合多轮，返回恶意客户端索引、余弦相似度矩阵
def foolsgold(roundsNum: List[int], dirPath: str) -> Tuple[np.ndarray, torch.Tensor]:
    clientPerRound = 50  # 这里就先写死了
    maliciousPerRound = 20
    gradients = [0] * len(roundsNum) * clientPerRound
    maliciouses = []
    for th, roundNum in enumerate(roundsNum):
        participants_thisRound = getParticipants(roundNum, dirPath)
        model_updates = loadPkl(roundNum, dirPath)
        keys = list(model_updates.keys())
        last_layer_updates = model_updates[keys[-2]]
        K = len(last_layer_updates)
        for i in range(K):
            thisParticipant = participants_thisRound[i]
            if thisParticipant < maliciousPerRound:
                thisIndex = th * maliciousPerRound + thisParticipant
            else:
                thisIndex = maliciousPerRound * len(roundsNum) + (clientPerRound - maliciousPerRound) * th + thisParticipant - maliciousPerRound
            gradients[thisIndex] = last_layer_updates[i].cpu().numpy()
        _, foolsgoldMaliciousIndex, _ = foolsgold_oneRound(roundNum, dirPath)
        for thisMaliciousIndex in foolsgoldMaliciousIndex:
            if thisMaliciousIndex < maliciousPerRound:
                thisIndex = th * maliciousPerRound + thisMaliciousIndex
            else:
                thisIndex = maliciousPerRound * len(roundsNum) + (clientPerRound - maliciousPerRound) * th + thisMaliciousIndex - maliciousPerRound
            maliciouses.append(thisIndex)
    shuffleArray = np.arange(50)
    np.random.shuffle(shuffleArray[0:len(roundsNum) * maliciousPerRound])
    np.random.shuffle(shuffleArray[len(roundsNum) * maliciousPerRound:50])
    malicious_shuffled = [shuffleArray[i] for i in maliciouses]
    gradients_shuffled = [gradients[shuffleArray[i]] for i in range(50)]
    cs = smp.cosine_similarity(gradients_shuffled)
    cs_tensor = torch.from_numpy(cs)
    return malicious_shuffled, cs_tensor


In [31]:
# ByGPT - 还是不太行啊看来
# import os
# import pickle
# import torch
# import torch.nn.functional as F
# import numpy as np
# from sklearn.cluster import KMeans
# from typing import List, Dict, Tuple


# def fltrust_original(model_updates: Dict[str, torch.Tensor], param_updates: List[torch.Tensor], clean_param_update: torch.Tensor) -> Dict[str, torch.Tensor]:
#     """
#     使用 FLTrust 方法进行聚合计算。
    
#     参数:
#     - model_updates: 客户端模型更新的字典，键为参数名，值为模型更新的 Tensor。
#     - param_updates: 客户端的参数更新列表，包含每个客户端的更新 Tensor。
#     - clean_param_update: 干净模型的参数更新，用于计算客户端更新的权重。

#     返回:
#     - global_update: 聚合后的全局模型更新。
#     """
#     cos = torch.nn.CosineSimilarity(dim=0)
#     g0_norm = torch.norm(clean_param_update)
#     weights = []
    
#     # 计算每个客户端更新与 clean_param_update 的余弦相似度
#     for param_update in param_updates:
#         weights.append(F.relu(cos(param_update.view(-1, 1), clean_param_update.view(-1, 1))))
    
#     weights = torch.tensor(weights).to('cuda:0').view(1, -1)
#     weights = weights / weights.sum()
#     weights = torch.where(weights[0].isnan(), torch.zeros_like(weights), weights)
#     nonzero_weights = torch.count_nonzero(weights.flatten())
#     nonzero_indices = torch.nonzero(weights.flatten()).flatten()

#     print(f'g0_norm: {g0_norm}, '
#           f'weights_sum: {weights.sum()}, '
#           f'*** {nonzero_weights} *** model updates are considered to be aggregated !')

#     normalize_weights = []
#     for param_update in param_updates:
#         normalize_weights.append(g0_norm / torch.norm(param_update))

#     global_update = dict()
#     for name, params in model_updates.items():
#         if 'num_batches_tracked' in name or 'running_mean' in name or 'running_var' in name:
#             global_update[name] = 1 / nonzero_weights * params[nonzero_indices].sum(dim=0, keepdim=True)
#         else:
#             global_update[name] = torch.matmul(
#                 weights,
#                 params * torch.tensor(normalize_weights).to('cuda:0').view(-1, 1))
#     return global_update


# def fltrust(roundsNum: List[int], dirPath: str, modelPath: str) -> Tuple[List[int], np.ndarray]:
#     """
#     根据指定的轮次和目录路径，使用 FLTrust 聚合和识别恶意客户端。

#     参数:
#     - roundsNum: 需要处理的轮次列表。
#     - dirPath: 数据文件所在的目录路径。
#     - modelPath: 模型路径（目前没有用到）。

#     返回:
#     - 恶意客户端的编号列表。
#     - 50 个客户端的评分（1x50 数组或 50x50 数组）。
#     """
    
#     # 加载pkl文件数据
#     def loadPkl(roundNum: int, subfolder: str, dirPath: str) -> Dict[str, torch.Tensor]:
#         pklPrefix = os.path.basename(os.path.normpath(dirPath)).split('-')[-1]
#         pklName = os.path.join(dirPath, f'{subfolder}/{pklPrefix}_{roundNum}.pkl')
#         with open(pklName, 'rb') as f:
#             update: Dict[str, torch.Tensor] = pickle.load(f)
#         return update
    
#     # 合并指定轮次的数据
#     all_model_updates = {}
#     all_param_updates = []
#     for roundNum in roundsNum:
#         participants_thisRound = getParticipants(roundNum, dirPath)
#         model_updates = loadPkl(roundNum, 'model_updates', dirPath)  # 修正为正确的子文件夹
#         clean_param_update = loadPkl(roundNum, 'clean_param_updates', dirPath)[participants_thisRound[0]]  # 修正为正确的子文件夹

#         # 拼接恶意客户端（0-2号）和良性客户端（3-9号）
#         for key, value in model_updates.items():
#             if key not in all_model_updates:
#                 all_model_updates[key] = torch.zeros((50, value.shape[1])).to('cuda:0')
#             all_model_updates[key][:15] = torch.cat([value[i].unsqueeze(0) for i in range(3)], dim=0).to('cuda:0')  # 恶意客户端
#             all_model_updates[key][15:] = torch.cat([value[i].unsqueeze(0) for i in range(3, 10)], dim=0).to('cuda:0')  # 良性客户端

#         # 提取参数更新，拼接恶意和良性客户端
#         for i in range(3):
#             all_param_updates.append(parameters_to_vector([model_updates[key][i] for key in model_updates.keys()]).to('cuda:0'))
#         for i in range(3, 10):
#             all_param_updates.append(parameters_to_vector([model_updates[key][i] for key in model_updates.keys()]).to('cuda:0'))
    
#     # 使用 FLTrust 进行聚合
#     global_update = fltrust_original(all_model_updates, all_param_updates, clean_param_update)
    
#     # 使用 KMeans 聚类来识别恶意客户端
#     scores = torch.stack(all_param_updates).cpu().numpy()  # 使用参数更新作为聚类的输入
#     kmeans = KMeans(n_clusters=2, random_state=0).fit(scores)
#     cluster_labels = kmeans.labels_
    
#     # 识别恶意客户端
#     malicious_clients = [i for i in range(50) if cluster_labels[i] == cluster_labels[:15].max()]

#     # 计算 50x50 的余弦相似度矩阵
#     cosine_similarity_matrix = np.zeros((50, 50))
#     for i in range(50):
#         for j in range(50):
#             cosine_similarity_matrix[i, j] = F.cosine_similarity(all_param_updates[i].view(-1, 1), all_param_updates[j].view(-1, 1)).item()

#     return malicious_clients, cosine_similarity_matrix


# # 辅助函数：获取参数更新的向量
# def parameters_to_vector(params):
#     return torch.cat([p.view(-1) for p in params])


# # 辅助函数：获取参与者列表
# def getParticipants(roundNum: int, dirPath: str) -> np.ndarray:
#     # 实现略
#     pass

In [32]:
# fltrust
# https://github.com/LetMeFly666/SecFFT/blob/706bb287c3b00f6143e2190edc74714ed88f3532/getResult/FL_Backdoor_CV/roles/aggregation_rules.py#L301-L340

def fltrust_original(model_updates, param_updates, clean_param_update):
    cos = torch.nn.CosineSimilarity(dim=0)
    g0_norm = torch.norm(clean_param_update)
    weights = []
    for param_update in param_updates:
        weights.append(F.relu(cos(param_update.view(-1, 1), clean_param_update.view(-1, 1))))
    weights = torch.tensor(weights).to('cuda:0').view(1, -1)
    weights = weights / weights.sum()
    weights = torch.where(weights[0].isnan(), torch.zeros_like(weights), weights)
    nonzero_weights = torch.count_nonzero(weights.flatten())
    nonzero_indices = torch.nonzero(weights.flatten()).flatten()

    print(f'g0_norm: {g0_norm}, '
          f'weights_sum: {weights.sum()}, '
          f'*** {nonzero_weights} *** model updates are considered to be aggregated !')

    normalize_weights = []
    for param_update in param_updates:
        normalize_weights.append(g0_norm / torch.norm(param_update))

    global_update = dict()
    for name, params in model_updates.items():
        if 'num_batches_tracked' in name or 'running_mean' in name or 'running_var' in name:
            global_update[name] = 1 / nonzero_weights * params[nonzero_indices].sum(dim=0, keepdim=True)
        else:
            global_update[name] = torch.matmul(
                weights,
                params * torch.tensor(normalize_weights).to('cuda:0').view(-1, 1))
    return global_update

def fltrust(roundsNum: List[int], dirPath: str, modelPath: str):
    clientPerRound = 50  # 这里就先写死了
    maliciousPerRound = 20
    # gradients = [0] * len(roundsNum) * clientPerRound
    # maliciouses = []
    gradientsList = []  # 里面存放每一轮的梯度，最后再聚合
    for th, roundNum in enumerate(roundsNum):
        participants_thisRound = getParticipants(roundNum, dirPath)
        # get model updates
        pklName = f'../NormalRun/FL_Backdoor_CV/saved_models/Revision_1/fltrust_NEUROTOXIN_09141511-fmnist/fltrust_{roundNum}.pth'
        # print(pklName)
        with open(pklName, 'rb') as f:
            model_updates: Dict[str, torch.Tensor] = torch.load(f)
        print(model_updates)
        pklName = f'../NormalRun/FL_Backdoor_CV/resultWithTime/2024-09-14_15-11-15-fltrust-fmnist_NEUROTOXIN/model_updates/fmnist_NEUROTOXIN_{roundNum}.pkl'
        with open(pklName, 'rb') as f:
            param_updates: Dict[str, torch.Tensor] = pickle.load(f)
        param_updates = torch.cat(list(param_updates.values()), dim=1)
        # print(param_updates[list(param_updates.keys())[0]].shape)  # torch.Size([10, 12288])
        pklName = f'../NormalRun/FL_Backdoor_CV/resultWithTime/2024-09-14_15-11-15-fltrust-fmnist_NEUROTOXIN/clean_param_updates/fmnist_NEUROTOXIN_{roundNum}.pkl'
        with open(pklName, 'rb') as f:
            clean_param_update: torch.Tensor = pickle.load(f)
        
        # global_update = fltrust_original(model_updates, [param_updates[key] for key in param_updates.keys()], clean_param_update)
        global_update = fltrust_original(model_updates, param_updates, clean_param_update)

        # print(clean_param_update.shape)  # torch.Size([2674688])
        thisGradients = [0] * clientPerRound


        gradientsList.append(thisGradients)
        break
        

In [33]:
def loadPkl(roundNum: int, dirPath: str) -> Dict[str, torch.Tensor]:  # 把pkl变成{'key1': [[user3], [user5]]}
    # dirPath = '../NormalRun/FL_Backdoor_CV/resultWithTime/2024-09-12_23-26-43-avg-fmnist_NEUROTOXIN/'
    pklPrefix = os.path.basename(os.path.normpath(dirPath)).split('-')[-1]
    pklName = os.path.join(dirPath, f'model_updates/{pklPrefix}_{roundNum}.pkl')
    with open(pklName, 'rb') as f:
        update: Dict[str, torch.Tensor] = pickle.load(f)
    return update

def getParticipants(roundNum: int, dirPath: str) -> np.ndarray:  # 获取参与者列表
    participantFilePath = os.path.join(dirPath, 'participants/participants.csv')
    participants = np.genfromtxt(
        participantFilePath, delimiter=",", dtype=None, encoding="utf-8"
    )
    participants = participants[1:].T
    participants_thisRound = participants[roundNum]  # 获取当前轮次的参与者
    return participants_thisRound

# # 通过历史记录的梯度计算恶意客户端
# def calc_maliciousAndCos_justByGrads(roundsNum: List[int], dirPath: str):
#     maliciousUpdates = []
#     benignUpdates = []
#     for th, roundNum in enumerate(roundsNum):
#         update = loadPkl(roundNum, dirPath)
#         update = torch.cat(list(update.values()), dim=1)
#         participants = getParticipants(roundNum, dirPath)
#         temp = torch.zeros(update.shape)
#         # print(update.shape)  # torch.Size([10, 2674688])
#         # print(temp.shape)
#         for i in range(10):
#             if participants[i] < 3:
#                 maliciousUpdates.append(update[i].cpu().numpy())
#             else:
#                 benignUpdates.append(update[i].cpu().numpy())
#     all = maliciousUpdates + benignUpdates
#     cs = cosine_similarity(all)
#     return cs

def kmeans_clustering(updates_np, n_clusters=2):
    return 0, [], 0
    # 使用KMeans算法进行聚类
    kmeans = KMeans(n_clusters=n_clusters, random_state=0)
    kmeans.fit(updates_np)

    # 获取聚类标签
    labels = kmeans.labels_
    
    # 识别被认为是恶意客户端的索引（假设第一个聚类为恶意客户端）
    malicious_indices = np.where(labels == 0)[0] if n_clusters == 2 else []

    return labels, malicious_indices, kmeans

# 通过历史记录的梯度计算恶意客户端
def calc_maliciousAndCos_justByGrads(roundsNum: List[int], dirPath: str):
    clientPerRound = 50  # 这里就先写死了
    maliciousPerRound = 20
    gradients = [0] * len(roundsNum) * clientPerRound
    maliciouses = []
    for th, roundNum in enumerate(roundsNum):
        participants_thisRound = getParticipants(roundNum, dirPath)
        model_updates = loadPkl(roundNum, dirPath)
        keys = list(model_updates.keys())
        last_layer_updates = model_updates[keys[-2]]
        K = len(last_layer_updates)
        for i in range(K):
            thisParticipant = participants_thisRound[i]
            if thisParticipant < maliciousPerRound:
                thisIndex = th * maliciousPerRound + thisParticipant
            else:
                thisIndex = maliciousPerRound * len(roundsNum) + (clientPerRound - maliciousPerRound) * th + thisParticipant - maliciousPerRound
            gradients[thisIndex] = last_layer_updates[i].cpu().numpy()
        # _, foolsgoldMaliciousIndex, _ = kmeans_clustering(gradients)
        _, foolsgoldMaliciousIndex, _ = foolsgold_oneRound(roundNum, dirPath)
        for thisMaliciousIndex in foolsgoldMaliciousIndex:
            if thisMaliciousIndex < maliciousPerRound:
                thisIndex = th * maliciousPerRound + thisMaliciousIndex
            else:
                thisIndex = maliciousPerRound * len(roundsNum) + (clientPerRound - maliciousPerRound) * th + thisMaliciousIndex - maliciousPerRound
            maliciouses.append(thisIndex)
    shuffleArray = np.arange(50)
    np.random.shuffle(shuffleArray[0:len(roundsNum) * maliciousPerRound])
    np.random.shuffle(shuffleArray[len(roundsNum) * maliciousPerRound:50])
    malicious_shuffled = [shuffleArray[i] for i in maliciouses]
    gradients_shuffled = [gradients[shuffleArray[i]] for i in range(50)]
    cs = smp.cosine_similarity(gradients_shuffled)
    cs_tensor = torch.from_numpy(cs)
    return malicious_shuffled, cs_tensor

def min_enclosing_ball(points, zeta, eta_prime, max_iter, lambda_thresh):
    """
    使用迭代方法计算最小覆盖超球的球心和半径。
    
    参数:
    - points: 客户端的历史梯度更新，形状为 (T, d)
    - zeta: 覆盖比例
    - eta_prime: 学习率
    - max_iter: 最大迭代次数
    - lambda_thresh: 收敛阈值
    
    返回:
    - center: 球心（意图点）
    - radius: 球半径
    """
    # 确保 points 是二维数组
    points = np.array(points)
    if len(points.shape) == 1:
        points = points.reshape(1, -1)
    
    # 初始化球心为所有点的均值
    O_i = np.mean(points, axis=0)
    # 初始化球半径为当前球心到所有点的最大距离
    r_i = max(np.linalg.norm(O_i - points, axis=1))
    
    # 迭代优化球心和半径
    for k in range(max_iter):
        # 计算球心到每个点的方向向量
        projection_points = []
        for point in points:
            direction = O_i - point
            norm_direction = np.linalg.norm(direction)
            if norm_direction == 0:
                proj_point = point  # 如果球心和点重合，直接使用点作为投影点
            else:
                # 投影点的计算
                proj_point = O_i + eta_prime * (point - O_i) / norm_direction
            projection_points.append(proj_point)
        
        # 确保 projection_points 不为空
        if len(projection_points) == 0:
            break
        
        # 计算投影点到球心的距离
        projection_points = np.array(projection_points)
        distances = np.linalg.norm(O_i - projection_points, axis=1)

        # 如果 selected_indices 为空，使用最大距离的点作为替代
        if len(distances) == 0:
            continue
        
        # 选择前 zeta*T 个投影点
        select_count = max(1, int(zeta * len(projection_points)))  # 确保至少选择一个点
        selected_indices = np.argsort(distances)[:select_count]
        
        if len(selected_indices) == 0:
            max_proj_point = projection_points[np.argmax(distances)]  # 使用最大距离的点
        else:
            max_proj_point = projection_points[selected_indices[-1]]  # 最远的投影点
        
        # 更新球心
        O_i = O_i + eta_prime * (max_proj_point - O_i)
        # 计算新的半径
        r_new = max(np.linalg.norm(O_i - projection_points[selected_indices], axis=1))
        
        # 检查收敛条件
        if abs(r_new - r_i) < lambda_thresh:
            break
        
        r_i = r_new
    
    return O_i, r_i

def calc_maliciousAndCos_justByGrads_SecFFT(roundsNum: List[int], dirPath: str, zeta=0.8, eta_prime=0.1, max_iter=100, lambda_thresh=1e-4):
    clientPerRound = 50  # 这里就先写死了
    maliciousPerRound = 20
    gradients = [0] * len(roundsNum) * clientPerRound
    # maliciouses = []
    for th, roundNum in enumerate(roundsNum):
        participants_thisRound = getParticipants(roundNum, dirPath)
        model_updates = loadPkl(roundNum, dirPath)
        keys = list(model_updates.keys())
        last_layer_updates = model_updates[keys[-2]]
        K = len(last_layer_updates)
        for i in range(K):
            thisParticipant = participants_thisRound[i]
            if thisParticipant < maliciousPerRound:
                thisIndex = th * maliciousPerRound + thisParticipant
            else:
                thisIndex = maliciousPerRound * len(roundsNum) + (clientPerRound - maliciousPerRound) * th + thisParticipant - maliciousPerRound
            gradients[thisIndex] = last_layer_updates[i].cpu().numpy()
        # _, foolsgoldMaliciousIndex, _ = kmeans_clustering(gradients)
        # _, foolsgoldMaliciousIndex, _ = foolsgold_oneRound(roundNum, dirPath)
        # for thisMaliciousIndex in foolsgoldMaliciousIndex:
        #     if thisMaliciousIndex < maliciousPerRound:
        #         thisIndex = th * maliciousPerRound + thisMaliciousIndex
        #     else:
        #         thisIndex = maliciousPerRound * len(roundsNum) + (clientPerRound - maliciousPerRound) * th + thisMaliciousIndex - maliciousPerRound
        #     maliciouses.append(thisIndex)
    shuffleArray = np.arange(50)
    np.random.shuffle(shuffleArray[0:len(roundsNum) * maliciousPerRound])
    np.random.shuffle(shuffleArray[len(roundsNum) * maliciousPerRound:50])
    # malicious_shuffled = [shuffleArray[i] for i in maliciouses]
    gradients_shuffled = [gradients[shuffleArray[i]] for i in range(50)]
    intentions = []
    for grad in gradients_shuffled:
        grad = np.array(grad)
        # 使用最小覆盖球算法计算意图点
        center, radius = min_enclosing_ball(grad, zeta, eta_prime, max_iter, lambda_thresh)
        intentions.append(center)  # 意图点是球心
    # print(intentions)
    # print(len(intentions))
    # for n_neighbors in range(1, 52):
    #     lof = LocalOutlierFactor(n_neighbors=n_neighbors, metric='euclidean')
    #     lof_scores = lof.fit_predict(intentions)  # 使用LOF检测异常客户端
    #     detected_malicious = [i for i, score in enumerate(lof_scores) if score == -1]
    #     print(f'n_neighbors = {n_neighbors}, len(detected_malicious) = {len(detected_malicious)}')
    # n_neighbors = 1, len(detected_malicious) = 0
    # n_neighbors = 2, len(detected_malicious) = 0
    # n_neighbors = 3, len(detected_malicious) = 0
    # n_neighbors = 4, len(detected_malicious) = 0
    # n_neighbors = 5, len(detected_malicious) = 0
    # n_neighbors = 6, len(detected_malicious) = 0
    # n_neighbors = 7, len(detected_malicious) = 0
    # n_neighbors = 8, len(detected_malicious) = 0
    # n_neighbors = 9, len(detected_malicious) = 0
    # n_neighbors = 10, len(detected_malicious) = 0
    # n_neighbors = 11, len(detected_malicious) = 0
    # n_neighbors = 12, len(detected_malicious) = 0
    # n_neighbors = 13, len(detected_malicious) = 0
    # n_neighbors = 14, len(detected_malicious) = 0
    # n_neighbors = 15, len(detected_malicious) = 15
    # n_neighbors = 16, len(detected_malicious) = 15
    # n_neighbors = 17, len(detected_malicious) = 15
    # n_neighbors = 18, len(detected_malicious) = 15
    # n_neighbors = 19, len(detected_malicious) = 15
    # n_neighbors = 20, len(detected_malicious) = 15
    # n_neighbors = 21, len(detected_malicious) = 15
    # n_neighbors = 22, len(detected_malicious) = 15
    # n_neighbors = 23, len(detected_malicious) = 15
    # n_neighbors = 24, len(detected_malicious) = 15
    # n_neighbors = 25, len(detected_malicious) = 15
    # n_neighbors = 26, len(detected_malicious) = 15
    # n_neighbors = 27, len(detected_malicious) = 15
    # n_neighbors = 28, len(detected_malicious) = 15
    # n_neighbors = 29, len(detected_malicious) = 15
    # n_neighbors = 30, len(detected_malicious) = 15
    # n_neighbors = 31, len(detected_malicious) = 15
    # n_neighbors = 32, len(detected_malicious) = 15
    # n_neighbors = 33, len(detected_malicious) = 15
    # n_neighbors = 34, len(detected_malicious) = 15
    # n_neighbors = 35, len(detected_malicious) = 0
    # n_neighbors = 36, len(detected_malicious) = 0
    # n_neighbors = 37, len(detected_malicious) = 0
    # n_neighbors = 38, len(detected_malicious) = 0
    # n_neighbors = 39, len(detected_malicious) = 0
    # n_neighbors = 40, len(detected_malicious) = 0
    # n_neighbors = 41, len(detected_malicious) = 0
    # n_neighbors = 42, len(detected_malicious) = 0
    # n_neighbors = 43, len(detected_malicious) = 0
    # n_neighbors = 44, len(detected_malicious) = 0
    # n_neighbors = 45, len(detected_malicious) = 0
    # n_neighbors = 46, len(detected_malicious) = 0
    # n_neighbors = 47, len(detected_malicious) = 0
    # n_neighbors = 48, len(detected_malicious) = 0
    # n_neighbors = 49, len(detected_malicious) = 0
    # n_neighbors = 50, len(detected_malicious) = 0
    # WC!!!!识别效果还真不错！！！
    lof = LocalOutlierFactor(n_neighbors=int(50 * 0.4), metric='euclidean')
    lof_scores = lof.fit_predict(intentions)  # 使用LOF检测异常客户端
    detected_malicious = [i for i, score in enumerate(lof_scores) if score == -1]
    print(detected_malicious)
    cs = cosine_similarity(intentions)
    cs_tensor = torch.from_numpy(cs)
    return detected_malicious, cs_tensor

In [34]:
def limit_attack_size(gradients, scaling_factor=0.1):
    """
    限制前 20 个恶意客户端的梯度更新大小。
    
    参数:
    - gradients: 包含 50 个客户端的梯度更新列表，每个元素是一个 NumPy 数组，表示一个客户端的梯度更新。
    - scaling_factor: 用于限制恶意客户端梯度大小的缩放因子。默认值为 0.1。
    
    返回:
    - new_gradients: 经过处理后的梯度更新列表，前 20 个恶意客户端的更新被限制大小。
    """
    # 确保输入是一个包含50个客户端梯度的列表
    assert len(gradients) == 50, "梯度列表的长度应该为 50"
    
    # 复制原始梯度列表以避免修改原始数据
    new_gradients = gradients.copy()
    
    # 限制前15个恶意客户端的梯度更新大小
    for i in range(20):  # 前15个是恶意客户端
        # 获取当前客户端的梯度更新
        original_grad = new_gradients[i]
        # 使用缩放因子限制梯度大小
        limited_grad = original_grad * scaling_factor
        # 更新新的梯度
        new_gradients[i] = limited_grad
    
    return new_gradients

# def limit_attack_direction(gradients, mixing_factor=0.1):
#     """
#     限制前 15 个恶意客户端的梯度方向，使其与正常客户端的梯度方向更相似。
    
#     参数:
#     - gradients: 包含 50 个客户端的梯度更新列表，每个元素是一个 NumPy 数组，表示一个客户端的梯度更新。
#     - mixing_factor: 用于混合恶意客户端梯度方向和正常客户端平均梯度方向的因子。默认值为 0.9。
    
#     返回:
#     - new_gradients: 经过处理后的梯度更新列表，前 15 个恶意客户端的更新被调整方向。
#     """
#     # 确保输入是一个包含50个客户端梯度的列表
#     assert len(gradients) == 50, "梯度列表的长度应该为 50"
    
#     # 复制原始梯度列表以避免修改原始数据
#     new_gradients = gradients.copy()
    
#     # 计算正常客户端的平均梯度方向（客户端 16 到 50）
#     normal_gradients = np.array(new_gradients[15:])  # 获取正常客户端的梯度
#     average_normal_gradient = np.mean(normal_gradients, axis=0)  # 计算平均梯度方向
    
#     # 对平均梯度进行归一化
#     average_normal_gradient /= np.linalg.norm(average_normal_gradient)
    
#     # 调整前 15 个恶意客户端的梯度方向
#     for i in range(15):  # 前15个是恶意客户端
#         # 获取当前恶意客户端的梯度更新
#         original_grad = new_gradients[i]
        
#         # 对原始梯度进行归一化
#         original_grad /= np.linalg.norm(original_grad)
        
#         # 将恶意客户端的梯度方向调整为与正常客户端的平均梯度方向更相似
#         adjusted_grad = mixing_factor * average_normal_gradient + (1 - mixing_factor) * original_grad
        
#         # 重新调整梯度的大小为原始大小
#         adjusted_grad *= np.linalg.norm(gradients[i])
        
#         # 更新新的梯度
#         new_gradients[i] = adjusted_grad
    
#     return new_gradients

def limit_attack_direction(gradients, mixing_factor=0.4):
    """
    限制前 15 个恶意客户端的梯度方向，使其随机向一个良性客户端的梯度方向偏移。
    
    参数:
    - gradients: 包含 50 个客户端的梯度更新列表，每个元素是一个 NumPy 数组，表示一个客户端的梯度更新。
    - mixing_factor: 用于混合恶意客户端梯度方向和随机选择的良性客户端梯度方向的因子。默认值为 0.9。
    
    返回:
    - new_gradients: 经过处理后的梯度更新列表，前 15 个恶意客户端的更新被调整方向。
    """
    # 确保输入是一个包含50个客户端梯度的列表
    assert len(gradients) == 50, "梯度列表的长度应该为 50"
    
    # 复制原始梯度列表以避免修改原始数据
    new_gradients = gradients.copy()
    
    # 对前 20 个恶意客户端进行处理
    for i in range(20):  # 前20个是恶意客户端
        # 随机选择一个良性客户端（20到50的索引是20到49）
        random_benign_index = np.random.randint(20, 50)
        benign_grad = new_gradients[random_benign_index]
        
        # 获取当前恶意客户端的梯度更新
        original_grad = new_gradients[i]
        
        # 对恶意客户端和选中的良性客户端的梯度进行归一化
        original_grad_normalized = original_grad / np.linalg.norm(original_grad)
        benign_grad_normalized = benign_grad / np.linalg.norm(benign_grad)
        
        # 将恶意客户端的梯度方向调整为向良性客户端方向偏移
        adjusted_grad = mixing_factor * benign_grad_normalized + (1 - mixing_factor) * original_grad_normalized
        
        # 重新调整梯度的大小为原始大小
        adjusted_grad *= np.linalg.norm(gradients[i])
        
        # 更新新的梯度
        new_gradients[i] = adjusted_grad
    
    return new_gradients

def identify_malicious_clients(cosine_similarity_tensor, threshold=5):
    """
    识别余弦相似度矩阵中的恶意客户端。
    
    参数:
    cosine_similarity_tensor (torch.Tensor): 50x50的余弦相似度张量
    threshold (float): 用于层次聚类的距离阈值
    
    返回:
    List[int]: 恶意客户端的下标
    """
    # 将 PyTorch 张量转换为 NumPy 数组
    data = cosine_similarity_tensor.numpy()

    # 对称化矩阵（确保余弦相似度矩阵是对称的）
    data = (data + data.T) / 2

    # 数据标准化
    scaler = StandardScaler()
    data_scaled = scaler.fit_transform(data)

    # 计算层次聚类的链接矩阵
    Z = linkage(data_scaled, method='ward')

    # 根据距离阈值对客户端进行聚类
    clusters = fcluster(Z, t=threshold, criterion='distance')

    # 找出每个簇的大小
    cluster_counts = np.bincount(clusters)

    # 识别恶意客户端所在的簇（假设簇中客户端数量较少的为恶意客户端）
    malicious_clusters = np.where(cluster_counts <= 2)[0]  # 这里我们假设小于等于2个客户端的簇为恶意簇

    # 找出恶意客户端的下标
    malicious_clients = [index for index, cluster_label in enumerate(clusters) if cluster_label in malicious_clusters]

    return malicious_clients

# 通过历史记录的梯度计算恶意客户端
def calc_maliciousAndCos_justByGrads_limited(roundsNum: List[int], dirPath: str, limistedMethod=0):  # limistedMethod: 0-限制大小，1-限制方向
    clientPerRound = 50  # 这里就先写死了
    maliciousPerRound = 20
    gradients = [0] * len(roundsNum) * clientPerRound
    for th, roundNum in enumerate(roundsNum):
        participants_thisRound = getParticipants(roundNum, dirPath)
        model_updates = loadPkl(roundNum, dirPath)
        keys = list(model_updates.keys())
        last_layer_updates = model_updates[keys[-2]]
        K = len(last_layer_updates)
        for i in range(K):
            thisParticipant = participants_thisRound[i]
            if thisParticipant < maliciousPerRound:
                thisIndex = th * maliciousPerRound + thisParticipant
            else:
                thisIndex = maliciousPerRound * len(roundsNum) + (clientPerRound - maliciousPerRound) * th + thisParticipant - maliciousPerRound
            gradients[thisIndex] = last_layer_updates[i].cpu().numpy()
    shuffleArray = np.arange(50)
    np.random.shuffle(shuffleArray[0:len(roundsNum) * maliciousPerRound])
    np.random.shuffle(shuffleArray[len(roundsNum) * maliciousPerRound:50])
    gradients_shuffled = [gradients[shuffleArray[i]] for i in range(50)]
    if limistedMethod == 0:
        gradients_limited = limit_attack_size(gradients_shuffled)
    else:
        gradients_limited = limit_attack_direction(gradients_shuffled)
    cs = smp.cosine_similarity(gradients_limited)
    malicious = identify_malicious_clients(torch.from_numpy(cs)) 
    cs_tensor = torch.from_numpy(cs)
    return malicious, cs_tensor

def calc_maliciousAndCos_justByGrads_limited_SecFFT(roundsNum: List[int], dirPath: str, limistedMethod=0, zeta=0.8, eta_prime=0.1, max_iter=100, lambda_thresh=1e-4):
    clientPerRound = 50  # 这里就先写死了  再不写死就写死了
    maliciousPerRound = 20
    gradients = [0] * len(roundsNum) * clientPerRound
    for th, roundNum in enumerate(roundsNum):
        participants_thisRound = getParticipants(roundNum, dirPath)
        model_updates = loadPkl(roundNum, dirPath)
        keys = list(model_updates.keys())
        last_layer_updates = model_updates[keys[-2]]
        K = len(last_layer_updates)
        for i in range(K):
            thisParticipant = participants_thisRound[i]
            if thisParticipant < maliciousPerRound:
                thisIndex = th * maliciousPerRound + thisParticipant
            else:
                thisIndex = maliciousPerRound * len(roundsNum) + (clientPerRound - maliciousPerRound) * th + thisParticipant - maliciousPerRound
            gradients[thisIndex] = last_layer_updates[i].cpu().numpy()
    shuffleArray = np.arange(50)
    np.random.shuffle(shuffleArray[0:len(roundsNum) * maliciousPerRound])
    np.random.shuffle(shuffleArray[len(roundsNum) * maliciousPerRound:50])
    # malicious_shuffled = [shuffleArray[i] for i in maliciouses]
    gradients_shuffled = [gradients[shuffleArray[i]] for i in range(50)]
    if limistedMethod == 0:
        gradients_limited = limit_attack_size(gradients_shuffled)
    else:
        gradients_limited = limit_attack_direction(gradients_shuffled)
    intentions = []
    for grad in gradients_limited:
        grad = np.array(grad)
        # 使用最小覆盖球算法计算意图点
        center, radius = min_enclosing_ball(grad, zeta, eta_prime, max_iter, lambda_thresh)
        intentions.append(center)  # 意图点是球心
    # lof = LocalOutlierFactor(n_neighbors=int(50 * 0.4), metric='euclidean')
    # lof_scores = lof.fit_predict(intentions)  # 使用LOF检测异常客户端
    # detected_malicious = [i for i, score in enumerate(lof_scores) if score == -1]
    # print(detected_malicious)
    cs = cosine_similarity(intentions)
    cs_tensor = torch.from_numpy(cs)
    return cs_tensor

In [35]:
def getScores(identified_malicious: List[int]) -> Tuple:
    total_clients = 50
    malicious_clients = {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19}  # 实际恶意客户端的索引

    # 计算TP, FP, TN, FN
    TP = len(malicious_clients.intersection(identified_malicious))
    FP = len(set(identified_malicious) - malicious_clients)
    FN = len(malicious_clients - set(identified_malicious))
    TN = total_clients - TP - FP - FN

    # 准确率（Accuracy）
    accuracy = (TP + TN) / total_clients

    # 精确率（Precision）
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0

    # 召回率（Recall）
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0

    # 特异度（Specificity）
    specificity = TN / (TN + FP) if (TN + FP) > 0 else 0

    # 误报率（False Positive Rate, FPR）
    fpr = FP / (FP + TN) if (FP + TN) > 0 else 0

    # 假阴率（False Negative Rate, FNR）
    fnr = FN / (TP + FN) if (TP + FN) > 0 else 0

    # 计算AUC（此处假设预测的概率值，以便计算AUC）
    y_true = [1 if i in malicious_clients else 0 for i in range(total_clients)]
    y_scores = [1 if i in identified_malicious else 0 for i in range(total_clients)]
    auc = roc_auc_score(y_true, y_scores)

    # 计算MCC（Matthews Correlation Coefficient）
    mcc = matthews_corrcoef(y_true, y_scores)

    return TP, FP, TN, FN, accuracy, precision, recall, specificity, fpr, fnr, auc, mcc

In [36]:
def plotOnce(plotRound: int=24):
    # malicious_clients, cosine_similarity_matrix = fltrust([15, 16, 17, 18, 19], '../NormalRun/FL_Backdoor_CV/resultWithTime/2024-09-14_15-11-15-fltrust-fmnist_NEUROTOXIN', '../NormalRun/FL_Backdoor_CV/saved_models/Revision_1/fltrust_NEUROTOXIN_09141511-fmnist')
    foolsgoldMaliciousIndex, foolsgoldScore2 = calc_maliciousAndCos_justByGrads([plotRound], 'D:/Data-R1/2024-10-28_05-59-22-foolsgold-fmnist_NEUROTOXIN')
    fltrustMaliciousIndex, fltrustScore2 = calc_maliciousAndCos_justByGrads([plotRound], 'D:/Data-R1/2024-10-03_06-48-00-fltrust-fmnist_NEUROTOXIN')
    # flameMaliciousIndex, flameMaliciousIndex = calc_maliciousAndCos_justByGrads([15, 16, 17, 18, 19], '../NormalRun/FL_Backdoor_CV/resultWithTime/2024-09-12_23-30-57-flame-fmnist_NEUROTOXIN')
    flameMaliciousIndex, flameScore2 = calc_maliciousAndCos_justByGrads([plotRound], 'D:/Data-R1/2024-10-02_12-55-32-flame-fmnist_NEUROTOXIN')
    secfftMaliciousIndex, secfftScore2 = calc_maliciousAndCos_justByGrads_SecFFT([plotRound], 'D:/Data-R1/2024-10-30_08-40-55-secfft-fmnist_NEUROTOXIN')

    foolsMalicious, foolsLimited = calc_maliciousAndCos_justByGrads_limited([plotRound], 'D:/Data-R1/2024-10-28_05-59-22-foolsgold-fmnist_NEUROTOXIN')
    fltrustMalicious, fltrustLimited = calc_maliciousAndCos_justByGrads_limited([plotRound], 'D:/Data-R1/2024-10-03_06-48-00-fltrust-fmnist_NEUROTOXIN')
    flameMalicious, flameLimited = calc_maliciousAndCos_justByGrads_limited([plotRound], 'D:/Data-R1/2024-10-02_12-55-32-flame-fmnist_NEUROTOXIN')
    secfftLimited = calc_maliciousAndCos_justByGrads_limited_SecFFT([plotRound], 'D:/Data-R1/2024-10-30_08-40-55-secfft-fmnist_NEUROTOXIN')


    foolsMalicious2, foolsDirection = calc_maliciousAndCos_justByGrads_limited([plotRound], 'D:/Data-R1/2024-10-28_05-59-22-foolsgold-fmnist_NEUROTOXIN', limistedMethod=1)
    fltrustMalicious2, fltrustDirection = calc_maliciousAndCos_justByGrads_limited([plotRound], 'D:/Data-R1/2024-10-03_06-48-00-fltrust-fmnist_NEUROTOXIN', limistedMethod=1)
    flameMalicious2, flameDirection = calc_maliciousAndCos_justByGrads_limited([plotRound], 'D:/Data-R1/2024-10-02_12-55-32-flame-fmnist_NEUROTOXIN', limistedMethod=1)
    secffDirection = calc_maliciousAndCos_justByGrads_limited_SecFFT([plotRound], 'D:/Data-R1/2024-10-30_08-40-55-secfft-fmnist_NEUROTOXIN', limistedMethod=1)

    # foolsgoldScore2 = calc_maliciousAndCos_justByGrads([15, 16, 17, 18, 19], '../NormalRun/FL_Backdoor_CV/resultWithTime/2024-09-13_23-15-48-foolsgold-fmnist_NEUROTOXIN')

    # print(foolsgoldMaliciousIndex)
    # print(foolsgoldScore2)
    # datas = [foolsgoldScore2] * 12
    # datas = [secfftScore2] * 12
    # fltrustScore2 -= np.eye(50)
    # fltrustLimited -= np.eye(50)
    # fltrustDirection -= np.eye(50)
    # fltrustScore2 -= np.eye(50) * fltrustScore2.min()
    # m = fltrustScore2.min()
    # fltrustScore2 = fltrustScore2.cpu().numpy()
    # np.fill_diagonal(fltrustScore2, m)
    # m = fltrustLimited.min()
    # fltrustLimited = fltrustLimited.cpu().numpy()
    # np.fill_diagonal(fltrustLimited, m)
    # m = fltrustDirection.min()
    # fltrustDirection = fltrustDirection.cpu().numpy()
    # np.fill_diagonal(fltrustDirection, m)
    datas = [foolsgoldScore2, fltrustScore2, flameScore2, secfftScore2,
            foolsLimited, fltrustLimited, flameLimited, secfftLimited,
            foolsDirection, fltrustDirection, flameDirection, secffDirection,]
    foolsgoldMaliciousIndex = [int(x) for x in foolsgoldMaliciousIndex]
    fltrustMaliciousIndex = [int(x) for x in fltrustMaliciousIndex]
    flameMaliciousIndex = [int(x) for x in foolsgoldMaliciousIndex]
    foolsgoldMaliciousIndex.sort()
    fltrustMaliciousIndex.sort()
    flameMaliciousIndex.sort()
    print(foolsgoldMaliciousIndex)
    print(fltrustMaliciousIndex)
    print(flameMaliciousIndex)
    print(secfftMaliciousIndex)

    print('----------------')
    print(foolsMalicious)
    print(fltrustMalicious)
    print(flameMalicious)

    print('----------------')
    print(foolsMalicious2)
    print(fltrustMalicious2)
    print(flameMalicious2)
    print('----------------')

    foolsgoldScores = getScores(foolsgoldMaliciousIndex)
    fltrustScores = getScores(fltrustMaliciousIndex)
    flameScores = getScores(flameMaliciousIndex)
    secfftScores = getScores(secfftMaliciousIndex)
    print(foolsgoldScores)
    print(fltrustScores)
    print(flameScores)
    print(secfftScores)

    plot_detection_heatmaps_3x4(*datas)


"""
我在jupyter里有一个plotOnce函数，这个函数会执行`plt.savefig`并保存一张图
我想调用这个函数25次，如何将25张图拼接到一块并显示到jupyter上？
```
for i in range(25):
    plotOnce(i)
```
这样的话只会显示最后一张，并且后面的图片会覆盖前面的图片



注意，我的plotOnce函数会执行`plt.savefig`并保存一张图到文件中，我不能修改这个函数。


进行如下修改：
1. 不要绘制成5x5的图像，而绘制成25x1的图像（每张图像都很长，一行只放置一个原始图像）
2. 每次调用plotOnce函数都会生成`detection_comparison_results_3x4.pdf`，第二次生成的文件会覆盖第一次的文件
"""
# save_dir = 'temp_saved_plots'  # 自定义保存路径
# os.makedirs(save_dir, exist_ok=True)
# for i in range(25):
#     plotOnce(i)
#     os.rename('detection_comparison_results_3x4.png', os.path.join(save_dir, f'detection_comparison_results_3x4_{i}.png'))
#     fig, axs = plt.subplots(25, 1, figsize=(10, 40))  # 25x1 的布局
# for i in range(25):
#     ax = axs[i]  # 确定当前子图的轴
#     img_path = os.path.join(save_dir, f'detection_comparison_results_3x4_{i}.png')
#     # images = convert_from_path(img_path)
#     # img = images[0]
#     img = Image.open(img_path)
#     ax.imshow(img)
#     ax.axis('off')  # 不显示坐标轴

# # 调整布局
# plt.tight_layout()
# plt.show()


save_dir = 'temp_saved_plots'  # 自定义保存路径
os.makedirs(save_dir, exist_ok=True)
for i in range(25):
    plotOnce(i)
    os.rename('detection_comparison_results_3x4.pdf', os.path.join(save_dir, f'detection_comparison_results_3x4_{i}.pdf'))

# 我决定不这么拼接了。我已经有了25个PDF，请将其拼接成一个PDF的25页
from PyPDF2 import PdfMerger
save_dir = 'temp_saved_plots'  # 假设所有 PDF 文件都在这个目录下
output_pdf_path = 'merged_output.pdf'  # 输出合并后的 PDF 文件名

# 创建 PdfMerger 对象
merger = PdfMerger()

# 遍历目录中的所有 PDF 文件
for i in range(25):
    pdf_path = os.path.join(save_dir, f'detection_comparison_results_3x4_{i}.pdf')
    merger.append(pdf_path)  # 将 PDF 文件添加到合并器中

# 合并并保存最终的 PDF 文件
merger.write(output_pdf_path)
merger.close()

print(f"合并完成，输出文件: {output_pdf_path}")

# 不错的有：9

识别出的恶意客户端索引: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         1.         1.         0.9408687  0.90504749
 1.         1.         0.51391395 1.         0.90504749 0.9408687
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         0.51391395 1.         1.         1.
 1.         1.        ]
识别出的恶意客户端索引: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         1.         1.         0.9408687  0.90504749
 1.         1.         0.51391395 1.         0.90504749 0.9408687
 1.         1.         1.    

c:\Users\admin\Desktop\LLM\wb2\Codes\getResult\genReLiTu.py:73: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, axes = plt.subplots(nrows=3, ncols=4, figsize=(20, 15))


识别出的恶意客户端索引: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
[1.         0.83281746 1.         1.         1.         1.
 1.         0.83281746 1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.        ]
识别出的恶意客户端索引: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
[0.91230211 1.         1.         1.         1.         1.
 1.         0.91230211 1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
